In [ ]:
def play(player1, player2):
    while player1.value > 0 and player2.value > 0:
        pass
        #player1.play(player2)

In [74]:
from random import choice

class Agent():
    both_cooperate = 1
    both_cheat = -1
    a_cheats = (1, -1)
    b_cheats = (-1, 1)
    num_agents = 0
    
    def __init__(self, val=10):
        self.val = val
        self.rounds = 0
        self.history = []
        Agent.num_agents += 1
        self.agent_id = Agent.num_agents
        self.name = 'Agent'
    
    @property
    def value(self):
        return self.val
    @property
    def rounds_played(self):
        return self.rounds
    @property
    def view_history(self):
        return self.history
    
    @classmethod
    def set_cooperate_amount(cls, amount):
        cls.both_cooperate = amount
    @classmethod
    def set_both_cheat_amount(cls, amount):
        cls.both_cheat = amount
    @classmethod
    def set_a_cheat_amount(cls, amount):
        amount = sorted(amount)
        cls.a_cheats = (amount[1], amount[0])
        cls.b_cheats = (amount[0], amount[1])
     
    def play(self, other):
        self.rounds += 1
        other.rounds += 1
        self_action = self.action(other)
        other_action = other.action(self)
        self.history.append({'opponent_id': other.agent_id, 'opponent_name': other.name,
                             'opponent_action': other_action, 'action': self_action})
        other.history.append({'opponent_id': self.agent_id, 'opponent_name': self.name,
                              'opponent_action': self_action, 'action': other_action})
        if self_action == 'cooperate' and other_action == 'cooperate':
            self.val += Agent.both_cooperate
            other.val += Agent.both_cooperate
        elif self_action == 'cheat' and other_action == 'cooperate':
            self.val += Agent.a_cheats[0]
            other.val += Agent.a_cheats[1]
        elif self_action == 'cooperate' and other_action == 'cheat':
            self.val += Agent.b_cheats[0]
            other.val += Agent.b_cheats[1]
        elif self_action == 'cheat' and other_action == 'cheat':
            self.val += Agent.both_cheat
            other.val += Agent.both_cheat

In [89]:
class Cooperator(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Cooperator'
        
    def action(self, opponent):
        act = 'cooperate'
        return act
    
class Cheater(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Cheater'
        
    def action(self, opponent):
        act = 'cheat'
        return act
    
class Random(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Random'
        
    def action(self, opponent):
        act = choice(['cheat', 'cooperate'])
        return act
    
class Tit_for_tat(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Tit for tat'
        
    def action(self, opponent):
        # may no longer need try/except block
        try:
            for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    if i['action'] == 'cheat':
                        return 'cheat'
            return 'cooperate'
        except:
            print('error, need to keep this try/except block')
            return 'cooperate'

class Suspicious_tit_for_tat(Agent):
    def __init__(self):
        super().__init__()
        self.name = 'Suspicious tit for tat'
        
    def action(self, opponent):
        faced_opponent = False
        for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    faced_opponent = True
                    break
        if faced_opponent == False:
            return 'cheat'
        # may no longer need try/except block
        try:
            for i in opponent.history[::-1]:
                if i['opponent_id'] == self.agent_id:
                    if i['action'] == 'cheat':
                        return 'cheat'
            return 'cooperate'
        except:
            print('error, need to keep this try/except block')
            return 'cooperate'

In [76]:
g = Cooperator()
k = Cooperator()
l = Cheater()

In [77]:
for i in range(3):
    g.play(k)
g.value, k.value

(13, 13)

In [59]:
g = Cooperator()
l = Cheater()
for i in range(3):
    g.play(l)
g.value, l.value

(7, 13)

In [60]:
l = Cheater()
m = Tit_for_tat()

for i in range(3):
    l.play(m)
m.value, l.value, m.history

cheated
cheated


(7,
 9,
 [(6, 'Cheater', 'cheat', 'cooperate'),
  (6, 'Cheater', 'cheat', 'cheat'),
  (6, 'Cheater', 'cheat', 'cheat')])

In [44]:
g = Cooperator()
l = Cheater()
for i in range(3):
    l.play(g)
g.value, l.value

(7, 13)

In [45]:
g = Cooperator()
l = Random()
for i in range(3):
    l.play(g)
g.value, l.value, l.history

(9,
 13,
 [(8, 'Cooperator', 'cooperate'),
  (8, 'Cooperator', 'cheat'),
  (8, 'Cooperator', 'cheat')])

In [90]:
k = Cooperator()
l = Cheater()
m = Tit_for_tat()
n = Suspicious_tit_for_tat()

In [91]:
for player in [k,l,m,n]:
    for player2 in [k,l,m,n]:
        if player != player2:
            player.play(player2)

In [92]:
n.history

[{'action': 'cheat',
  'opponent_action': 'cooperate',
  'opponent_id': 17,
  'opponent_name': 'Cooperator'},
 {'action': 'cheat',
  'opponent_action': 'cheat',
  'opponent_id': 18,
  'opponent_name': 'Cheater'},
 {'action': 'cheat',
  'opponent_action': 'cooperate',
  'opponent_id': 19,
  'opponent_name': 'Tit for tat'},
 {'action': 'cooperate',
  'opponent_action': 'cooperate',
  'opponent_id': 17,
  'opponent_name': 'Cooperator'},
 {'action': 'cheat',
  'opponent_action': 'cheat',
  'opponent_id': 18,
  'opponent_name': 'Cheater'},
 {'action': 'cooperate',
  'opponent_action': 'cheat',
  'opponent_id': 19,
  'opponent_name': 'Tit for tat'}]